### XGBoost



- XGBoost is also known as ‘regularized boosting‘ technique which helps to reduce overfitting.
- XGBoost implements parallel processing and is blazingly faster as compared to GBM.
- XGBoost allow users to define custom optimization objectives and evaluation criteria.
- XGBoost has an in-built routine to handle missing values.
- XGBoost make splits upto the max_depth specified and then start pruning the tree backwards and remove splits beyond which there is no positive gain.
- XGBoost allows user to run a cross-validation at each iteration of the boosting process.

<img src="xgboost.png">

Link:
https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/


In [5]:
import pandas as pd, numpy as np, time
from sklearn.model_selection import train_test_split

from IPython.display import Image

import warnings
warnings.filterwarnings('ignore')

In [6]:
data = pd.read_csv("/home/manishv/workspace/data/flights.csv")
data = data.sample(frac = 0.1, random_state=10)

In [7]:
data.head(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
411984,2015,1,28,3,WN,103,N7728D,DCA,MKE,705,...,811.0,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3591965,2015,8,11,2,B6,153,N592JB,JFK,PBI,1859,...,345.0,337.0,0,0,NaN,0.0,0.0,82.0,255.0,0.0
526451,2015,2,4,3,DL,1187,N921DN,MSP,DCA,1735,...,2043.0,-19.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1336011,2015,3,27,5,WN,171,N407WN,DEN,RDU,1815,...,2313.0,-7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3424502,2015,8,1,6,WN,4330,N7751A,ATL,RIC,2125,...,2318.0,13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
data = data[["MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT","AIR_TIME", "DEPARTURE_TIME","DISTANCE","ARRIVAL_DELAY"]]
data.dropna(inplace=True)

data["ARRIVAL_DELAY"] = (data["ARRIVAL_DELAY"]>10)*1

cols = ["AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT","ORIGIN_AIRPORT"]
for item in cols:
    data[item] = data[item].astype("category").cat.codes +1

data.head(5)

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,DESTINATION_AIRPORT,ORIGIN_AIRPORT,AIR_TIME,DEPARTURE_TIME,DISTANCE,ARRIVAL_DELAY
411984,1,28,3,14,102,717,608,102.0,713.0,634,0
3591965,8,11,2,3,152,748,690,134.0,111.0,1028,1
526451,2,4,3,4,1184,597,740,111.0,1734.0,931,0
1336011,3,27,5,14,170,770,609,173.0,1807.0,1436,0
3424502,8,1,6,14,4321,772,544,63.0,2151.0,481,1


In [9]:
train, test, y_train, y_test = train_test_split(data.drop(["ARRIVAL_DELAY"], axis=1), data["ARRIVAL_DELAY"],
                                                random_state=10, test_size=0.25)

### Model:



#### Parameters:

- General Parameters: Guide the overall functioning
     1. booster [default=gbtree]: Select the type of model to run at each iteration. 
        (a) gbtree: tree-based models 
        (b) gblinear: linear models
     2. silent [default=0]: Silent mode is activated is set to 1, i.e. no running messages will be printed.
     3. nthread [default to maximum number of threads available if not set]
  
  
- Booster Parameters: Guide the individual booster (tree/regression) at each step
     1. learning_rate ETA:Controls the contribution of weak learners in the final combination. There is a trade-off between learning_rate and n_estimators (It controls the number of weak learners). Eta [default=0.3] : learning rate and makes the model more robust by shrinking the weights on each step.
     2. min_child_weight [default=1] Defines the minimum sum of weights of all observations required in a child.Used to control over-fitting. Higher values prevent a model from learning relations which might be highly specific to the particular sample selected for a tree. Too high values can lead to under-fitting hence, it should be tuned using CV.
     3. max_depth: maximum depth of the individual regression estimators. The maximum depth limits the number of nodes in the tree. Tune this parameter for best performance; the best value depends on the interaction of the input variables. max_depth [default=6] maximum depth of a tree, same as GBM.
     4. max_leaf_nodes: The maximum number of terminal nodes or leaves in a tree.
     5. gamma [default=0]: A node is split only when the resulting split gives a positive reduction in the loss function. Gamma specifies the minimum loss reduction required to make a split.
     6. max_delta_step [default=0]: In maximum delta step we allow each tree’s weight estimation to be. If the value is set to 0, it means there is no constraint. If it is set to a positive value, it can help making the update step more conservative.
     7. subsample [default=1] : same as the subsample of GBM. Denotes the fraction of observations to be randomly samples for each tree.
     8. colsample_bytree [default=1] :Similar to max_features in GBM. Denotes the fraction of columns to be randomly samples for each tree.
     9. colsample_bylevel [default=1] :Denotes the subsample ratio of columns for each split, in each level.
     10. lambda [default=1]: L2 regularization term on weights
     11. alpha [default=0] : L1 regularization term on weight
     12. scale_pos_weight [default=1] : A value greater than 0 should be used in case of high class imbalance as it helps in faster convergence.



- Learning Task Parameters: Guide the optimization performed
    1. objective [default=reg:linear]: defines the loss function to be minimized. Mostly used values are:
    
    (a) binary:logistic –logistic regression for binary classification, returns predicted probability (not class)
    
    (b) multi:softmax –multiclass classification using the softmax objective, returns predicted class (not probabilities)you also need to set an additional num_class (number of classes) parameter defining the number of unique classes
    
    (c) multi:softprob –same as softmax, but returns predicted probability of each data point belonging to each class.

    2. eval_metric: The metric to be used for validation data. The default values are rmse for regression and error for classification.

    (a) rmse – root mean square error
    
    (b) mae – mean absolute error
    
    (c) logloss – negative log-likelihood
    
    (d) error – Binary classification error rate (0.5 threshold)
    
    (e) merror – Multiclass classification error rate
    
    (f) mlogloss – Multiclass logloss
    
    (g) auc: Area under the curve

    3. seed [default=0] : The random number seed.
    
 

In [18]:
def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

In [21]:
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import cross_val_predict

# Parameter Tuning
model = xgb.XGBClassifier()
param_dist = {"max_depth": [10,30,50],
              "min_child_weight" : [1,3,6],
              "n_estimators": [200],
              "learning_rate": [0.05, 0.1,0.16],}
rand_search = RandomizedSearchCV(model, param_distributions=param_dist, cv = 3, 
                                   verbose=10, n_jobs=-1)
rand_search.fit(train, y_train)



Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 29.8min
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed: 39.7min remaining: 23.0min
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed: 48.2min remaining: 14.7min
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed: 51.0min remaining:  5.7min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 56.3min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'n_estimators': [200], 'learning_rate': [0.05, 0.1, 0.16], 'max_depth': [10, 30, 50], 'min_child_weight': [1, 3, 6]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=10)

In [22]:
rand_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=30, min_child_weight=3, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [23]:
model = xgb.XGBClassifier(max_depth=50, min_child_weight=1,  n_estimators=200,\
                          n_jobs=-1 , verbose=1,learning_rate=0.16)
model.fit(train,y_train)

auc(model, train, test)

(0.9999999999999999, 0.7890576313373696)

### Automated hyperparameter tuning: BO

In [48]:
from sklearn.metrics import mean_absolute_error

from hyperopt import hp, tpe, STATUS_OK, Trials

col_train = train.columns

def xgb_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y), np.exp(yhat))

def objective(space):

    clf = xgb.XGBRegressor(n_estimators = space['n_estimators'],
                           max_depth = space['max_depth'],
                           min_child_weight = space['min_child_weight'],
                           subsample = space['subsample'],
                           learning_rate = space['learning_rate'],
                           gamma = space['gamma'],
                           colsample_bytree = space['colsample_bytree'],
                           objective='reg:linear'
                           )

    eval_set  = [(train, y_train), (test, y_test)]

    clf.fit(train[col_train],
            y_train,
            eval_set=eval_set,
            eval_metric = 'mae')

    pred = clf.predict(test)
    mae = mean_absolute_error((y_test), (pred))

#    print "SCORE:", mae
    return{'loss':mae, 'status': STATUS_OK }


space ={
        'max_depth': hp.choice('max_depth', np.arange(10, 30, dtype=int)),
        'min_child_weight': hp.quniform ('min_child', 1, 20, 1),
        'subsample': hp.uniform ('subsample', 0.8, 1),
        'n_estimators' : hp.choice('n_estimators', np.arange(1000, 10000, 100, dtype=int)),
        'learning_rate' : hp.quniform('learning_rate', 0.025, 0.5, 0.025),
        'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05)
    }


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=3, # change
            trials=trials)

print(best)

#{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.17500000000000002, 'n_estimators': 39, 
#'subsample': 0.8662635874308865, 'min_child': 16.0, 'max_depth': 3, 'gamma': 0.8500000000000001}

[0]	validation_0-mae:0.471802	validation_1-mae:0.473807
[1]	validation_0-mae:0.442496	validation_1-mae:0.447881
[2]	validation_0-mae:0.417247	validation_1-mae:0.425708
[3]	validation_0-mae:0.395148	validation_1-mae:0.406785
[4]	validation_0-mae:0.375951	validation_1-mae:0.390267
[5]	validation_0-mae:0.358721	validation_1-mae:0.375972
[6]	validation_0-mae:0.343651	validation_1-mae:0.363592
[7]	validation_0-mae:0.330903	validation_1-mae:0.353232
[8]	validation_0-mae:0.319846	validation_1-mae:0.343958
[9]	validation_0-mae:0.30988	validation_1-mae:0.335682
[10]	validation_0-mae:0.301537	validation_1-mae:0.329047
[11]	validation_0-mae:0.293425	validation_1-mae:0.32314
[12]	validation_0-mae:0.285587	validation_1-mae:0.317604
[13]	validation_0-mae:0.279189	validation_1-mae:0.313106
[14]	validation_0-mae:0.273434	validation_1-mae:0.309072
[15]	validation_0-mae:0.268497	validation_1-mae:0.305342
[16]	validation_0-mae:0.263919	validation_1-mae:0.301982
[17]	validation_0-mae:0.259936	validation_1

[144]	validation_0-mae:0.191994	validation_1-mae:0.272507
[145]	validation_0-mae:0.191952	validation_1-mae:0.272539
[146]	validation_0-mae:0.191879	validation_1-mae:0.27251
[147]	validation_0-mae:0.19162	validation_1-mae:0.272437
[148]	validation_0-mae:0.191539	validation_1-mae:0.272424
[149]	validation_0-mae:0.191417	validation_1-mae:0.272407
[150]	validation_0-mae:0.191375	validation_1-mae:0.27239
[151]	validation_0-mae:0.191122	validation_1-mae:0.272355
[152]	validation_0-mae:0.190978	validation_1-mae:0.27235
[153]	validation_0-mae:0.190696	validation_1-mae:0.272269
[154]	validation_0-mae:0.190628	validation_1-mae:0.272259
[155]	validation_0-mae:0.190479	validation_1-mae:0.272228
[156]	validation_0-mae:0.190214	validation_1-mae:0.272191
[157]	validation_0-mae:0.190056	validation_1-mae:0.272162
[158]	validation_0-mae:0.189948	validation_1-mae:0.272173
[159]	validation_0-mae:0.189795	validation_1-mae:0.272144
[160]	validation_0-mae:0.189731	validation_1-mae:0.272146
[161]	validation_0

[286]	validation_0-mae:0.177015	validation_1-mae:0.27091
[287]	validation_0-mae:0.177007	validation_1-mae:0.270911
[288]	validation_0-mae:0.176906	validation_1-mae:0.27091
[289]	validation_0-mae:0.176833	validation_1-mae:0.270914
[290]	validation_0-mae:0.17678	validation_1-mae:0.270914
[291]	validation_0-mae:0.176636	validation_1-mae:0.270886
[292]	validation_0-mae:0.17657	validation_1-mae:0.27089
[293]	validation_0-mae:0.176517	validation_1-mae:0.270902
[294]	validation_0-mae:0.176392	validation_1-mae:0.270904
[295]	validation_0-mae:0.176312	validation_1-mae:0.270918
[296]	validation_0-mae:0.17625	validation_1-mae:0.270913
[297]	validation_0-mae:0.17616	validation_1-mae:0.27092
[298]	validation_0-mae:0.176077	validation_1-mae:0.270894
[299]	validation_0-mae:0.176044	validation_1-mae:0.270887
[300]	validation_0-mae:0.176005	validation_1-mae:0.270886
[301]	validation_0-mae:0.175784	validation_1-mae:0.27084
[302]	validation_0-mae:0.17577	validation_1-mae:0.270846
[303]	validation_0-mae:0

[429]	validation_0-mae:0.168892	validation_1-mae:0.270335
[430]	validation_0-mae:0.168823	validation_1-mae:0.270332
[431]	validation_0-mae:0.168813	validation_1-mae:0.270333
[432]	validation_0-mae:0.168757	validation_1-mae:0.270368
[433]	validation_0-mae:0.168692	validation_1-mae:0.270367
[434]	validation_0-mae:0.168641	validation_1-mae:0.27037
[435]	validation_0-mae:0.168628	validation_1-mae:0.270379
[436]	validation_0-mae:0.168619	validation_1-mae:0.270384
[437]	validation_0-mae:0.168586	validation_1-mae:0.270371
[438]	validation_0-mae:0.168564	validation_1-mae:0.270366
[439]	validation_0-mae:0.168529	validation_1-mae:0.270371
[440]	validation_0-mae:0.168454	validation_1-mae:0.270354
[441]	validation_0-mae:0.168413	validation_1-mae:0.270361
[442]	validation_0-mae:0.168387	validation_1-mae:0.270356
[443]	validation_0-mae:0.168342	validation_1-mae:0.27037
[444]	validation_0-mae:0.168331	validation_1-mae:0.270372
[445]	validation_0-mae:0.168308	validation_1-mae:0.270348
[446]	validation

[571]	validation_0-mae:0.163679	validation_1-mae:0.270099
[572]	validation_0-mae:0.163609	validation_1-mae:0.27008
[573]	validation_0-mae:0.163584	validation_1-mae:0.270093
[574]	validation_0-mae:0.163553	validation_1-mae:0.270099
[575]	validation_0-mae:0.163502	validation_1-mae:0.27008
[576]	validation_0-mae:0.163447	validation_1-mae:0.270073
[577]	validation_0-mae:0.163381	validation_1-mae:0.270048
[578]	validation_0-mae:0.163377	validation_1-mae:0.270045
[579]	validation_0-mae:0.163328	validation_1-mae:0.270043
[580]	validation_0-mae:0.163306	validation_1-mae:0.270035
[581]	validation_0-mae:0.163272	validation_1-mae:0.270019
[582]	validation_0-mae:0.163208	validation_1-mae:0.269998
[583]	validation_0-mae:0.163192	validation_1-mae:0.269999
[584]	validation_0-mae:0.163172	validation_1-mae:0.269992
[585]	validation_0-mae:0.163122	validation_1-mae:0.269996
[586]	validation_0-mae:0.163078	validation_1-mae:0.270008
[587]	validation_0-mae:0.163066	validation_1-mae:0.270007
[588]	validation

...

[9109]	validation_0-mae:0.118643	validation_1-mae:0.284271
[9110]	validation_0-mae:0.118641	validation_1-mae:0.284266
[9111]	validation_0-mae:0.118643	validation_1-mae:0.284269
[9112]	validation_0-mae:0.118643	validation_1-mae:0.284269
[9113]	validation_0-mae:0.118645	validation_1-mae:0.284273
[9114]	validation_0-mae:0.118643	validation_1-mae:0.284271
[9115]	validation_0-mae:0.118648	validation_1-mae:0.284279
[9116]	validation_0-mae:0.118641	validation_1-mae:0.28428
[9117]	validation_0-mae:0.118641	validation_1-mae:0.28428
[9118]	validation_0-mae:0.118638	validation_1-mae:0.284273
[9119]	validation_0-mae:0.118633	validation_1-mae:0.284263
[9120]	validation_0-mae:0.118638	validation_1-mae:0.284273
[9121]	validation_0-mae:0.118636	validation_1-mae:0.284282
[9122]	validation_0-mae:0.118629	validation_1-mae:0.284277
[9123]	validation_0-mae:0.11863	validation_1-mae:0.284281
[9124]	validation_0-mae:0.118634	validation_1-mae:0.284288
[9125]	validation_0-mae:0.118636	validation_1-mae:0.284292


[9249]	validation_0-mae:0.118575	validation_1-mae:0.284278
[9250]	validation_0-mae:0.118574	validation_1-mae:0.284274
[9251]	validation_0-mae:0.118569	validation_1-mae:0.284265
[9252]	validation_0-mae:0.11857	validation_1-mae:0.284267
[9253]	validation_0-mae:0.118572	validation_1-mae:0.28427
[9254]	validation_0-mae:0.118572	validation_1-mae:0.284272
[9255]	validation_0-mae:0.118574	validation_1-mae:0.284275
[9256]	validation_0-mae:0.118577	validation_1-mae:0.284282
[9257]	validation_0-mae:0.118575	validation_1-mae:0.284278
[9258]	validation_0-mae:0.118574	validation_1-mae:0.284274
[9259]	validation_0-mae:0.118576	validation_1-mae:0.284279
[9260]	validation_0-mae:0.118571	validation_1-mae:0.284269
[9261]	validation_0-mae:0.118573	validation_1-mae:0.284274
[9262]	validation_0-mae:0.118575	validation_1-mae:0.284278
[9263]	validation_0-mae:0.118573	validation_1-mae:0.284274
[9264]	validation_0-mae:0.11857	validation_1-mae:0.284268
[9265]	validation_0-mae:0.11857	validation_1-mae:0.284267
[

[9389]	validation_0-mae:0.118533	validation_1-mae:0.284281
[9390]	validation_0-mae:0.118531	validation_1-mae:0.284276
[9391]	validation_0-mae:0.118533	validation_1-mae:0.284281
[9392]	validation_0-mae:0.118533	validation_1-mae:0.284281
[9393]	validation_0-mae:0.118536	validation_1-mae:0.284286
[9394]	validation_0-mae:0.118533	validation_1-mae:0.284281
[9395]	validation_0-mae:0.118528	validation_1-mae:0.284269
[9396]	validation_0-mae:0.118521	validation_1-mae:0.284266
[9397]	validation_0-mae:0.118527	validation_1-mae:0.284277
[9398]	validation_0-mae:0.118529	validation_1-mae:0.284282
[9399]	validation_0-mae:0.118527	validation_1-mae:0.284278
[9400]	validation_0-mae:0.118531	validation_1-mae:0.284285
[9401]	validation_0-mae:0.118535	validation_1-mae:0.284294
[9402]	validation_0-mae:0.118539	validation_1-mae:0.284303
[9403]	validation_0-mae:0.118536	validation_1-mae:0.284296
[9404]	validation_0-mae:0.118536	validation_1-mae:0.284296
[9405]	validation_0-mae:0.118534	validation_1-mae:0.2842

[9529]	validation_0-mae:0.11844	validation_1-mae:0.284284
[9530]	validation_0-mae:0.118439	validation_1-mae:0.284281
[9531]	validation_0-mae:0.118442	validation_1-mae:0.284288
[9532]	validation_0-mae:0.118448	validation_1-mae:0.284299
[9533]	validation_0-mae:0.118445	validation_1-mae:0.284294
[9534]	validation_0-mae:0.118451	validation_1-mae:0.284305
[9535]	validation_0-mae:0.118446	validation_1-mae:0.284296
[9536]	validation_0-mae:0.118449	validation_1-mae:0.284302
[9537]	validation_0-mae:0.11845	validation_1-mae:0.284304
[9538]	validation_0-mae:0.118451	validation_1-mae:0.284306
[9539]	validation_0-mae:0.118451	validation_1-mae:0.284306
[9540]	validation_0-mae:0.118449	validation_1-mae:0.284301
[9541]	validation_0-mae:0.118455	validation_1-mae:0.284313
[9542]	validation_0-mae:0.118452	validation_1-mae:0.284307
[9543]	validation_0-mae:0.11845	validation_1-mae:0.284304
[9544]	validation_0-mae:0.118453	validation_1-mae:0.284309
[9545]	validation_0-mae:0.11845	validation_1-mae:0.284303
[

[9669]	validation_0-mae:0.118372	validation_1-mae:0.284294
[9670]	validation_0-mae:0.118372	validation_1-mae:0.284296
[9671]	validation_0-mae:0.118376	validation_1-mae:0.284303
[9672]	validation_0-mae:0.118375	validation_1-mae:0.284301
[9673]	validation_0-mae:0.118378	validation_1-mae:0.284307
[9674]	validation_0-mae:0.118383	validation_1-mae:0.284317
[9675]	validation_0-mae:0.118379	validation_1-mae:0.28431
[9676]	validation_0-mae:0.118373	validation_1-mae:0.284297
[9677]	validation_0-mae:0.118362	validation_1-mae:0.284287
[9678]	validation_0-mae:0.118364	validation_1-mae:0.284292
[9679]	validation_0-mae:0.118362	validation_1-mae:0.284288
[9680]	validation_0-mae:0.118363	validation_1-mae:0.284289
[9681]	validation_0-mae:0.118363	validation_1-mae:0.28429
[9682]	validation_0-mae:0.118366	validation_1-mae:0.284295
[9683]	validation_0-mae:0.118361	validation_1-mae:0.284297
[9684]	validation_0-mae:0.118358	validation_1-mae:0.284291
[9685]	validation_0-mae:0.11836	validation_1-mae:0.284294


In [20]:
import xgboost as xgb
from sklearn import metrics
model_best = xgb.XGBClassifier(max_depth=3, 
                               min_child_weight=16.0,
                               colsample_bytree= 0.70,
                               subsample= 0.86,
                               n_jobs=-1,
                               n_estimators=39,
                               verbose=1,
                               learning_rate=0.175,
                               gamma= 0.85)
model_best.fit(train,y_train)

auc(model_best, train, test)

(0.68322491280598, 0.6795450003851986)